In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import copy
#conda install -c conda-forge cufflinks-py
#conda install plotly
import ipywidgets as wg
from IPython.display import display

import cufflinks as cf
import chart_studio.plotly as py
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import inspect
import seaborn as sns

from matplotlib import pyplot as plt
init_notebook_mode(connected=True)
cf.go_offline()
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, Normalizer, PolynomialFeatures
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction import FeatureHasher
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.base import TransformerMixin, BaseEstimator
pd.options.display.max_columns = 200
pd.options.display.max_rows = 272

In [2]:
from sklearn.feature_selection import SelectKBest, VarianceThreshold, RFE, SelectFromModel
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error, make_scorer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from joblib import dump, load
from Logger import RegressionLogger, FuncTransformer, ColSelect

In [3]:
import inspect

In [4]:
log_transformed = 0      

In [5]:
model_log = load("data/model_logging.joblib")

In [6]:
current_model = copy.deepcopy(model_log[0]["model"])

# Now Lets Load our original Data

In [7]:
df = pd.read_csv("data/cleaned_df.csv", index_col="UnitID")

In [8]:
df.drop("Unnamed: 0",axis=1, inplace=True)

In [9]:
df.drop(["City location of institution (HD2019)", "Institution Name"], axis=1, inplace=True)

In [10]:
df_black = df.dropna(subset=[df.columns[-7]])

In [11]:
X_black = df_black.iloc[:,:-14]
y_black = df_black.iloc[:, -7]

In [12]:
X_black = pd.get_dummies(X_black, drop_first=True)

In [36]:
X_black.dtypes

Core_Revenues                                                                                                                                                       float64
Tuition_And_Fees                                                                                                                                                    float64
Government_Grants                                                                                                                                                   float64
Private_Gifts                                                                                                                                                       float64
Investment_Return                                                                                                                                                   float64
Sales_And_Services                                                                                                                          

# Lets Define Our Pipeline

In [37]:
num_feats = 200
num_pipe = Pipeline([
    ('Impute', ColumnTransformer([
        ('Impute', KNNImputer(n_neighbors=5), make_column_selector(dtype_include='float'))
    ], remainder='drop'
    )),
    ('Scale', StandardScaler())
])

cat_pipe = Pipeline([
    
    ('Passthrough', ColumnTransformer([
        ('Pass', 'passthrough', make_column_selector(dtype_exclude='float'))
    ], remainder='drop'
    ))
])

preprocessor = FeatureUnion([
    ('numeric', num_pipe),
    ('categorical', cat_pipe)
])

pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', ElasticNet(alpha=0.1, l1_ratio=0.5))
])

In [38]:
params = {
    "Imputer": "SimpleImputer",
    "Scaler": "Standard Scaler",
    "model_name": "Elastic Net",
    "model": pipeline,
    "alpha": 0.1,
    "l1_ratio": 0.5,
}

In [39]:
logger = RegressionLogger(params)

In [40]:
logger.train_update(pipeline, X_black, y_black)

In [44]:
f'train and test mae: {logger.train_mae, logger.mae}'

'train and test mae: (13.400120158691642, 15.0942486907444)'